In [ ]:
import os, sys
import pickle   
from transformers import CamembertTokenizer
from torch.utils.data import DataLoader, Dataset
import torch
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
import torch.nn as nn
from datasets import load_from_disk
from tqdm import tqdm
import time

# 1. Prepare Data :

In [2]:
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

In [3]:
class Tokenized_oscar_dataset(Dataset):
    def __init__(self, raw_tokenised_data):
        """
        Initialiser le dataset avec les données sous forme de dictionnaire.
        """
        self.raw_data = raw_tokenised_data  # Le dictionnaire avec 'input_ids', 'attention_mask', et 'labels'
        self.data = {
            "texts": self.raw_data['text'],
            "input_ids": self.raw_data['input_ids'],
            "attention_mask": self.raw_data['attention_mask'],
            "labels": self.raw_data['labels']
        }
        
    def __len__(self):
        """
        Retourne le nombre d'exemples dans le dataset.
        """
        return len(self.data['input_ids'])
    
    def __getitem__(self, idx):
        """
        Retourne un exemple individuel sous forme de dictionnaire.
        """
        return {
            'input_ids': torch.tensor(self.data['input_ids'][idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.data['attention_mask'][idx], dtype=torch.long),
            'labels': torch.tensor(self.data['labels'][idx], dtype=torch.long)
        }

    
def mask_tokens(inputs, tokenizer, mlm_probability=0.15):
    """Prepare masked tokens for MLM."""
    labels = inputs.clone()
    probability_matrix = torch.full(labels.shape, mlm_probability)
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # Replace 80% of masked tokens with [MASK]
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # Replace 10% of masked tokens with random words
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    return inputs, labels


def preprocess_function(texts, tokenizer, max_length=512):
    """
    Préparer les données pour l'entraînement MLM.
    :param texts: Liste de textes.
    :param tokenizer: Tokenizer de CamemBERT.
    :return: Dictionnaire avec 'input_ids', 'attention_mask', et 'labels'.
    """
    # Tokeniser les textes
    tokenized = tokenizer(texts, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    # Appliquer le masquage
    masked_inputs, labels = mask_tokens(input_ids, tokenizer)

    return { 
        "text": list(texts),
        "input_ids": masked_inputs.tolist(),
        "attention_mask": attention_mask.tolist(),
        "labels": labels.tolist()
    }

load the downlaoded Oscar :

In [ ]:
dataset_path ="data/datasets/CamemBERT/data/CamemBERT/data/mini_oscar/mini_dataset.arrow"
mini_oscar_dataset = load_from_disk(dataset_path)
train_texts, val_texts = train_test_split(mini_oscar_dataset['text'], test_size=0.2, random_state=42)

In [ ]:
train_data = preprocess_function(train_texts, tokenizer)
val_data = preprocess_function(val_texts, tokenizer)

In [ ]:
import pickle

def save_tokenized_data(data, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "wb") as f:
        pickle.dump(data, f)

save_tokenized_data(train_data, "Noureddine/MLA-CamemBERT/data/Oscar/tokenized_train_data.pkl")
save_tokenized_data(val_data, "Noureddine/MLA-CamemBERT/data/Oscar/tokenized_val_data.pkl")

In [7]:
def load_tokenized_data(filename):
    with open(filename, "rb") as f:
        data = pickle.load(f)
    return data

loaded_train_data = load_tokenized_data("/home/amine/Noureddine/MLA-CamemBERT/data/Oscar/tokenized_train_data.pkl")
loaded_val_data = load_tokenized_data("/home/amine/Noureddine/MLA-CamemBERT/data/Oscar/tokenized_val_data.pkl")

# 2. Model :

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class CamembertConfig:
    def __init__(self):
        self.vocab_size = 32005
        self.hidden_size = 768
        self.num_hidden_layers = 12
        self.num_attention_heads = 12
        self.intermediate_size = 3072
        self.hidden_act = "gelu"
        self.hidden_dropout_prob = 0.1
        self.attention_probs_dropout_prob = 0.1
        self.max_position_embeddings = 514
        self.type_vocab_size = 1
        self.initializer_range = 0.02
        self.layer_norm_eps = 1e-5
        self.pad_token_id = 1
        self.head_type = "MLM"

class CamembertEmbeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids, token_type_ids=None, position_ids=None):
        input_shape = input_ids.size()
        seq_length = input_shape[1]

        if position_ids is None:
            position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device).unsqueeze(0)
        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=input_ids.device)

        inputs_embeds = self.word_embeddings(input_ids)
        position_embeds = self.position_embeddings(position_ids)
        token_type_embeds = self.token_type_embeddings(token_type_ids)

        embeddings = inputs_embeds + position_embeds + token_type_embeds
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)

        # Debug prints
        # print(f"Embeddings NaN: {torch.isnan(embeddings).any()}")

        return embeddings

class CamembertSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = config.hidden_size // config.num_attention_heads
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)
        self.dropout = nn.Dropout(0.2)  # Increased dropout rate

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        return x.view(new_x_shape).permute(0, 2, 1, 3)

    def forward(self, hidden_states, attention_mask=None):
        query_layer = self.transpose_for_scores(self.query(hidden_states))
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))

        # Debug query, key, value
        # print(f"Query NaN: {torch.isnan(query_layer).any()}")
        # print(f"Key NaN: {torch.isnan(key_layer).any()}")
        # print(f"Value NaN: {torch.isnan(value_layer).any()}")

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores /= math.sqrt(self.attention_head_size)

        # Clamp scores to prevent overflow
        attention_scores = torch.clamp(attention_scores, min=-1e9, max=1e9)
        attention_probs = nn.functional.softmax(attention_scores, dim=-1) + 1e-9
        attention_probs = self.dropout(attention_probs)

        # Debug attention scores and probabilities
        # print(f"Attention Scores NaN Before Clamp: {torch.isnan(attention_scores).any()}")
        # print(f"Attention Probs NaN: {torch.isnan(attention_probs).any()}")

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        context_layer = context_layer.view(context_layer.size(0), -1, self.all_head_size)

        # Debug context layer
        # print(f"Context Layer NaN: {torch.isnan(context_layer).any()}")

        return context_layer



class CamembertFeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.activation = F.gelu if config.hidden_act == "gelu" else nn.ReLU()
        self.dense_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(0.2)  # Increased dropout rate

    def forward(self, hidden_states):
        intermediate_output = self.activation(self.dense_1(hidden_states))
        intermediate_output = torch.clamp(intermediate_output, min=-1e9, max=1e9)

        output = self.dense_2(intermediate_output)
        output = self.dropout(output)
        output = self.LayerNorm(output + hidden_states)

        # Debug intermediate and final outputs
        # print(f"Intermediate Output NaN: {torch.isnan(intermediate_output).any()}")
        # print(f"Final Output NaN: {torch.isnan(output).any()}")

        return output


class CamembertLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attention = CamembertSelfAttention(config)
        self.attention_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.feed_forward = CamembertFeedForward(config)

    def forward(self, hidden_states, attention_mask=None):
        attention_output = self.attention(hidden_states, attention_mask)
        hidden_states = self.attention_norm(hidden_states + attention_output)
        return self.feed_forward(hidden_states)

class CamembertEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layers = nn.ModuleList([CamembertLayer(config) for _ in range(config.num_hidden_layers)])

    def forward(self, hidden_states, attention_mask=None):
        for i, layer in enumerate(self.layers):
            hidden_states = layer(hidden_states, attention_mask)

            # Debug prints for each layer
            # print(f"Layer {i} Hidden States NaN: {torch.isnan(hidden_states).any()}")

        return hidden_states

class CamembertLMHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.decoder = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

    def forward(self, hidden_states):
        hidden_states = F.gelu(self.dense(hidden_states))
        hidden_states = self.layer_norm(hidden_states)
        logits = self.decoder(hidden_states)

        # Debug prints
        # print(f"Logits NaN: {torch.isnan(logits).any()}")

        return logits

class CamembertModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = CamembertEmbeddings(config)
        self.encoder = CamembertEncoder(config)
        self.head = CamembertLMHead(config) if config.head_type == "MLM" else None

    def forward(self, input_ids, attention_mask=None):
        embedded_input = self.embeddings(input_ids)

        if attention_mask is not None:
            attention_mask = (1.0 - attention_mask) * -float('inf')

        encoder_output = self.encoder(embedded_input, attention_mask)
        return self.head(encoder_output)


In [18]:
def reinitialize_weights(module):
    if isinstance(module, (nn.Linear, nn.Embedding)):
        module.reset_parameters()
    elif isinstance(module, nn.LayerNorm):
        module.reset_parameters()
    elif hasattr(module, "weight") and module.weight is not None:
        nn.init.xavier_uniform_(module.weight)  # Xavier initialization
    if hasattr(module, "bias") and module.bias is not None:
        nn.init.zeros_(module.bias)


In [19]:
from torchinfo import summary 

config = CamembertConfig()
model = CamembertModel(config)
# # Reinitialize weights
model.apply(reinitialize_weights)
print("Model weights reinitialized successfully.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Model moved to GPU successfully.")
summary(model)

Model weights reinitialized successfully.
Model moved to GPU successfully.


Layer (type:depth-idx)                             Param #
CamembertModel                                     --
├─CamembertEmbeddings: 1-1                         --
│    └─Embedding: 2-1                              24,579,840
│    └─Embedding: 2-2                              394,752
│    └─Embedding: 2-3                              768
│    └─LayerNorm: 2-4                              1,536
│    └─Dropout: 2-5                                --
├─CamembertEncoder: 1-2                            --
│    └─ModuleList: 2-6                             --
│    │    └─CamembertLayer: 3-1                    6,497,280
│    │    └─CamembertLayer: 3-2                    6,497,280
│    │    └─CamembertLayer: 3-3                    6,497,280
│    │    └─CamembertLayer: 3-4                    6,497,280
│    │    └─CamembertLayer: 3-5                    6,497,280
│    │    └─CamembertLayer: 3-6                    6,497,280
│    │    └─CamembertLayer: 3-7                    6,497,280
│    │    └

# 3. Train :

In [11]:
def save_checkpoint(epoch, model, optimizer, loss, save_dir):
    """
    Sauvegarder l'état actuel du modèle, optimiseur et perte.
    """
    checkpoint = {
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "loss": loss,
    }
    save_path = os.path.join(save_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save(checkpoint, save_path)
    print(f"Checkpoint saved at {save_path}")

In [12]:
batch_size = 32
train_dataset = Tokenized_oscar_dataset(loaded_train_data)
val_dataset = Tokenized_oscar_dataset(loaded_val_data)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [13]:
# 2. model :
#from transformers import CamembertForMaskedLM
#model = CamembertForMaskedLM.from_pretrained("camembert-base")
#model.apply(init_weights)
#print()

training loop :

In [ ]:
import os
import time
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import get_scheduler

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = model.to(device)
print("Model moved to device.")

max_lr = 6e-4
min_lr = max_lr * 0.1  # 10 %
warmup_steps = 10
num_epochs = 10  # Total epochs
save_interval = 1  
eval_interval = 1  
learning_rate = 1e-4

# Optimizer and Scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = get_scheduler(
    "cosine", 
    optimizer=optimizer, 
    num_warmup_steps=warmup_steps, 
    num_training_steps=num_epochs * len(train_loader)
)

history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

# Save Directory
save_dir = "models/Pretraining/model_checkpoints"
os.makedirs(save_dir, exist_ok=True)
print(f"Model checkpoints will be saved in: {save_dir}")

############################## Utility Functions ################################
def compute_accuracy(predictions, labels, ignore_index=-100):
    mask = labels != ignore_index
    correct = (predictions[mask] == labels[mask]).sum().item()
    total = mask.sum().item()
    return correct / total if total > 0 else 0.0

############################## Training Loop ################################
for epoch in range(num_epochs):
    print(f"\n--- Starting Epoch {epoch + 1}/{num_epochs} ---")
    model.train()  # Training mode
    train_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    t0 = time.time()

    # Training Phase
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}/{num_epochs}") as pbar:
        for batch_index, batch in enumerate(train_loader):
            # Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), ignore_index=-100)

            # Backward pass
            loss.backward()
            optimizer.step()
            scheduler.step()  # Update learning rate

            # Track Loss and Accuracy
            train_loss += loss.item()
            predictions = logits.argmax(dim=-1)
            correct_predictions += compute_accuracy(predictions, labels)
            pbar.set_postfix({'loss': loss.item()})
            pbar.update(1)

            # Print batch loss every 50 batches
            if batch_index % 50 == 0:
                print(f"  [Batch {batch_index}/{len(train_loader)}] Loss: {loss.item():.4f}")

    avg_train_loss = train_loss / len(train_loader)
    train_acc = correct_predictions / len(train_loader)

    t1 = time.time()
    print(f"Epoch {epoch + 1} Completed. Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_acc:.4f}, Time: {t1 - t0:.2f}s")

    history['train_loss'].append(avg_train_loss)
    history['train_acc'].append(train_acc)

    # Validation Phase
    if (epoch + 1) % eval_interval == 0 or (epoch + 1) == num_epochs:
        model.eval()
        val_loss = 0.0
        correct_val_predictions = 0

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                logits = model(input_ids, attention_mask=attention_mask)
                loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), ignore_index=-100)
                val_loss += loss.item()

                # Accuracy
                predictions = logits.argmax(dim=-1)
                correct_val_predictions += compute_accuracy(predictions, labels)

            avg_val_loss = val_loss / len(val_loader)
            avg_val_acc = correct_val_predictions / len(val_loader)
            print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {avg_val_acc:.4f}")
            history['val_loss'].append(avg_val_loss)
            history['val_acc'].append(avg_val_acc)

    # Save Model Checkpoints
    if (epoch + 1) % save_interval == 0 or (epoch + 1) == num_epochs:
        print(f"Saving checkpoint at epoch {epoch + 1}...")
        save_checkpoint(epoch + 1, model, optimizer, avg_train_loss, save_dir)
        print(f"Checkpoint saved for epoch {epoch + 1}.")
        
print("\nTraining completed.")


Using device: cuda
Model moved to device.
Model checkpoints will be saved in: models/Pretraining/model_checkpoints

--- Starting Epoch 1/10 ---


Epoch 1/10:   0%|          | 1/16913 [00:00<4:16:46,  1.10it/s, loss=10.5]

  [Batch 0/16913] Loss: 10.5191


Epoch 1/10:   0%|          | 51/16913 [00:44<4:07:49,  1.13it/s, loss=7.88]

  [Batch 50/16913] Loss: 7.8792


...

Epoch 3/10:  94%|█████████▎| 15851/16913 [3:53:41<15:39,  1.13it/s, loss=7.66]

  [Batch 15850/16913] Loss: 7.6638


Epoch 3/10:  94%|█████████▍| 15901/16913 [3:54:25<14:54,  1.13it/s, loss=7.45]

  [Batch 15900/16913] Loss: 7.4526


Epoch 3/10:  94%|█████████▍| 15930/16913 [3:54:50<14:30,  1.13it/s, loss=7.79]

In [15]:
print(len(train_loader))

16913
